In [1]:
import sys, os
import argparse

import numpy as np
import pandas as pd
import uproot
import csv, yaml

import h5py
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
def nerror(eff,N):
    y = np.sqrt(((100-eff)*eff)/N)
    return y

In [3]:
# runnum = os.listdir('/store/hep/users/yewzzang/JSNS2_22y/com_data')

In [4]:
# path = '/store/hep/users/yewzzang/JSNS2_22y/com_data/'
# FN = '/FN_cut_Rho_1.4_ZL_1.0'
# ME = '/ME_cut_Rho_1.4_ZL_1.0'

In [5]:
# exist = []
# nofile = []
# runs = []
# for i in range(len(runnum)):
    
#     if os.path.exists(path+runnum[i]+FN):
#         exist.append(len(os.listdir(path+runnum[i]+FN)))
#         runs.append(runnum[i][3:])
#     else:
#         nofile.append(runnum[i])

In [6]:
runss =[ '1592']

In [7]:
run_num = []
FN_num = []
ME_num = []
FN_rej = []
ME_eff = []
AUC = []


if os.path.exists('result/date20220427_train_1592_noDIN/date20220427_train_1592_noDIN.txt') == True:
    f = open('result/date20220427_train_1592_noDIN/date20220427_train_1592_noDIN.txt', 'r')
    lines = f.readlines()
#     for line in lines:
#         print(line)
    run_num.append(j)
#         print(lines)
#         print(float(lines[-2].split(' ')[-2]))
#         print(float(lines[-2].split(' ')[-1][:-1]))
#         print(j)
    FN_num.append(float(lines[-2].split(' ')[-2]))        #### FN num
    ME_num.append(float(lines[-2].split(' ')[-1][:-1]))   #### ME num
    ME_eff.append(float(lines[0].split(' ')[-1][:-2])*100)  #### ME effi
    FN_rej.append((1-float(lines[1].split(' ')[-1][:-2]))*100)    #### FN rej
    AUC.append(float(lines[-1][:-2]))                       #### AUC
    f.close()


In [8]:
# a = 0
# c = 0
# sets = []
# setss = []
# for i in range(len(np.array(FN_num))):
    
#     b = np.array(FN_num)[i]
#     a = a + b
#     sets.append(np.array(runss)[i])
#     if a > 7000:
#         print(a,i, np.array(runss)[i])
#         c = c+1
#         a = 0
#         print(c,'c')
#         setss.append(sets)
#         sets = []

In [9]:


infos = []
sis = []

preds = []
labels = []
dVertexs = []
dTs = []
vertexXs = []
vertexYs = []
vertexZs = []

result_output = 'test0426'


result_input = 'date20220427_train_1563_noDIN'
print(result_input)

si_path = 'result/' + result_input + '/sampleInfo_eval.csv'
fPred = 'result/' + result_input + '/' + result_input + '.csv'
si = pd.read_csv(si_path)
info = pd.read_csv(fPred)

info_numpy = np.array(info)
si_numpy = np.array(si)


for i in range(len(info_numpy)):

    label = info_numpy[i][0]
    pred = info_numpy[i][1]

    fileidx = info_numpy[i][4]

    filename = si_numpy[int(fileidx)][2]


    idx = info_numpy[i][5]


    data = h5py.File(filename,'r')

    dT = data['events']['dT'][idx]
    dVertex = data['events']['dVertex'][idx]
    vertexX = data['events']['vertexX'][idx]
    vertexY = data['events']['vertexY'][idx]
    vertexZ = data['events']['vertexZ'][idx]


    preds.append(pred)
    labels.append(label)
    dVertexs.append(dVertex)
    dTs.append(dT)
#     print(dTs.type)
    vertexXs.append(vertexX)
    vertexYs.append(vertexY)
    vertexZs.append(vertexZ)





preds = np.array(preds)
labels = np.array(labels)
dVertexs = np.array(dVertexs)
dTs = np.array(dTs)
vertexXs = np.array(vertexXs)
vertexYs = np.array(vertexYs)
vertexZs = np.array(vertexZs)


ME_label = []
ME_dVertex = []
ME_dT = []
ME_vertexX = []
ME_vertexY = []
ME_vertexZ = []
ME_pred = []


FN_label = []
FN_dVertex = []
FN_dT = []
FN_vertexX = []
FN_vertexY = []
FN_vertexZ = []
FN_pred = []

for i in range(len(preds)):
    if labels[i] == 1:
        ME_label.append(labels[i])
        ME_dVertex.append(dVertexs[i])
        ME_dT.append(dTs[i])
        ME_vertexX.append(vertexXs[i])
        ME_vertexY.append(vertexYs[i])
        ME_vertexZ.append(vertexZs[i])
        ME_pred.append(preds[i])
    else:

        FN_label.append(labels[i])
        FN_dVertex.append(dVertexs[i])
        FN_dT.append(dTs[i])
        FN_vertexX.append(vertexXs[i])
        FN_vertexY.append(vertexYs[i])
        FN_vertexZ.append(vertexZs[i])
        FN_pred.append(preds[i])

ME_sig = -np.log((1/np.array(ME_pred))-1)
FN_sig = -np.log((1/np.array(FN_pred))-1)
FN_over_0 = 100*np.sum(FN_sig > 0)/len(FN_sig)
ME_under_0 = 100*np.sum(ME_sig < 0)/len(ME_sig)

list_range = np.arange(0,1,0.0001)


# for i in range(len(list_range)):
#     a = len(np.array(ME_pred)[np.array(ME_pred)>list_range[i]])/len(np.array(ME_pred))
#     b = len(np.array(FN_pred)[np.array(FN_pred)<list_range[i]])/len(np.array(FN_pred))
# #     print(list_range[i])

#     if (a > 0.99):

#         eff_99 = list_range[i]


#     if (a > 0.95):

#         eff_95 = list_range[i]


#     if (a > 0.90):

#         eff_90 = list_range[i]


# #     
#     if ((0.903 > b)&(b > 0.897)):

#         eff_90_range = list_range[i]


#     if ((0.953 > b) & (b > 0.947)):
# #         if (b < 0.95):

#         eff_95_range = list_range[i]


#     if ((0.993 > b) & (b > 0.987)):
# #         if (b < 0.99):
#         eff_99_range = list_range[i]

# ##### eff 0.5 pred
# eff_50_ME = len(np.array(ME_pred)[np.array(ME_pred)>0.5])/len(np.array(ME_pred))
# eff_50_FN = len(np.array(FN_pred)[np.array(FN_pred)>0.5])/len(np.array(FN_pred))
# num_50_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>0.5])/len(np.array(ME_pred))
# num_50_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>0.5])/len(np.array(FN_pred))



# ##### eff 99% efficient
# eff_99_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_99])/len(np.array(ME_pred))
# eff_99_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_99])/len(np.array(FN_pred))
# num_99_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_99])/len(np.array(ME_pred))
# num_99_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_99])/len(np.array(FN_pred))




# ##### eff 95% efficient
# eff_95_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_95])/len(np.array(ME_pred))
# eff_95_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_95])/len(np.array(FN_pred))
# num_95_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_95])/len(np.array(ME_pred))
# num_95_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_95])/len(np.array(FN_pred))


# ##### eff 90% efficient
# eff_90_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_90])/len(np.array(ME_pred))
# eff_90_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_90])/len(np.array(FN_pred))
# num_90_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_90])/len(np.array(ME_pred))
# num_90_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_90])/len(np.array(FN_pred))




# ##### eff 99%_FN efficient
# eff_99_ME_FN = len(np.array(ME_pred)[np.array(ME_pred)<eff_99_range])/len(np.array(ME_pred))
# eff_99_FN_FN = len(np.array(FN_pred)[np.array(FN_pred)<eff_99_range])/len(np.array(FN_pred))
# num_99_ME_FN = num_ME*len(np.array(ME_pred)[np.array(ME_pred)<eff_99_range])/len(np.array(ME_pred))
# num_99_FN_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)<eff_99_range])/len(np.array(FN_pred))


# ##### eff 95%_FN efficient
# eff_95_ME_FN = len(np.array(ME_pred)[np.array(ME_pred)<eff_95_range])/len(np.array(ME_pred))
# eff_95_FN_FN = len(np.array(FN_pred)[np.array(FN_pred)<eff_95_range])/len(np.array(FN_pred))
# num_95_ME_FN = num_ME*len(np.array(ME_pred)[np.array(ME_pred)<eff_95_range])/len(np.array(ME_pred))
# num_95_FN_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)<eff_95_range])/len(np.array(FN_pred))


# ##### eff 90%_FN efficient
# eff_90_ME_FN = len(np.array(ME_pred)[np.array(ME_pred)<eff_90_range])/len(np.array(ME_pred))
# eff_90_FN_FN = len(np.array(FN_pred)[np.array(FN_pred)<eff_90_range])/len(np.array(FN_pred))
# num_90_ME_FN = num_ME*len(np.array(ME_pred)[np.array(ME_pred)<eff_90_range])/len(np.array(ME_pred))
# num_90_FN_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)<eff_90_range])/len(np.array(FN_pred))




date20220427_train_1563_noDIN


/users/yewzzang/.local/lib/python3.7/site-packages/ipykernel_launcher.py:107: RuntimeWarning: divide by zero encountered in log
/users/yewzzang/.local/lib/python3.7/site-packages/ipykernel_launcher.py:108: RuntimeWarning: divide by zero encountered in log


In [28]:
list_range = 0.387
a = len(np.array(ME_pred)[np.array(ME_pred)>list_range])/len(np.array(ME_pred))
b = len(np.array(FN_pred)[np.array(FN_pred)<list_range])/len(np.array(FN_pred))

print(a,'ME')
print(b,'FN')

0.9365139980968599 ME
0.9213048855905999 FN


In [30]:
nerror(92.13,9100 )

0.28227161340854623